In [1]:
from utilities import *
from core.initial import Insertions
import pandas as pd

In [2]:
datafile = "D:/Google Drive/MSc MST-AUEB/_Thesis_/Main Thesis/Model Data.xlsx"
d = DataProvider(filepath=datafile, route='A2')
model = Model(d)
model.build_model()

In [3]:
sol = Insertions(model)

In [4]:
sol.solve()

In [5]:
sol.sol.trip_duty_arr

array([[1, 0, 0, ..., 0, 0, 0],
       [0, 0, 1, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]])

In [6]:
for duty in sol.duties:
    print(duty)

Duty(ID=1, start=330, end=444, trips=2, driving_time=113, rests=1 breaks=0, trips=1-3
Duty(ID=3, start=345, end=464, trips=2, driving_time=113, rests=1 breaks=0, trips=2-4
Duty(ID=5, start=365, end=489, trips=2, driving_time=123, rests=1 breaks=0, trips=3-5
Duty(ID=7, start=380, end=509, trips=2, driving_time=123, rests=1 breaks=0, trips=4-6
Duty(ID=9, start=395, end=524, trips=2, driving_time=123, rests=1 breaks=0, trips=5-7
Duty(ID=11, start=410, end=550, trips=2, driving_time=129, rests=1 breaks=0, trips=6-8
Duty(ID=13, start=435, end=575, trips=2, driving_time=134, rests=1 breaks=0, trips=7-10
Duty(ID=15, start=440, end=585, trips=2, driving_time=134, rests=1 breaks=0, trips=8-11
Duty(ID=17, start=450, end=595, trips=2, driving_time=134, rests=1 breaks=0, trips=9-12
Duty(ID=19, start=460, end=605, trips=2, driving_time=134, rests=1 breaks=0, trips=10-13
Duty(ID=21, start=470, end=615, trips=2, driving_time=134, rests=1 breaks=0, trips=11-14
Duty(ID=23, start=480, end=625, trips=2, 

In [ ]:
B = 9
w = [2,3,4,5,6,7,8]
q = [4,2,6,6,2,2,2]
s=[]

for j in range(len(w)):
    for i in range(q[j]):
        s.append(w[j])

print(s)
print(B)